In [1]:
import pandas as pd
import numpy as np
from ortools.linear_solver import pywraplp as OR

In [2]:
def redistrict(k, A, cost, integer=False, opt_type='abs_val', solver='CBC'):
    """A model for solving a congressional redistricting problem.
    
    Args:
        k (int): number of districts in a plan
        A (np.ndarray): binary matrix a_ij = 1 if tract i is in district j
        costs (np.ndarray): cost coefficients of districts
        opt_type (str): {"minimize", "maximize", "abs_val"}
        solver (str) : {"CBC", "gurobi"}
    """
    n_tracts, n_columns = A.shape
    TRACTS = range(n_tracts)
    DISTRICTS = range(n_columns)
    
    # define the model
    if solver=='CBC':
        m = OR.Solver('redistrict', OR.Solver.CBC_MIXED_INTEGER_PROGRAMMING)        
    elif solver=='gurobi':
        m = OR.Solver('redistrict', OR.Solver.GUROBI_MIXED_INTEGER_PROGRAMMING)
    else:
        raise ValueError('Invalid solver')

    # decision variables
    x = {} # x_i is 1 if district i is used, 0 otherwise
    for d in DISTRICTS:
        if integer:
            x[d] = m.IntVar(0, 1, name="x(%s)" % d)
        else:
            x[d] = m.NumVar(0, 1, name="x(%s)" % d)

    # objective function
    if opt_type == 'min':
        m.Minimize(sum(cost[d] * x[d] for d in DISTRICTS))
    elif opt_type == 'max':
        m.Maximize(sum(cost[d] * x[d] for d in DISTRICTS))
    elif opt_type == 'abs_val':
        w = m.NumVar(-k, k, name="w")
        m.Add(sum(cost[d] * x[d] for d in DISTRICTS) <= w, name='absval_pos')
        m.Add(sum(cost[d] * x[d] for d in DISTRICTS) >= -w, name='absval_neg')
        m.Minimize(w)
    else:
        raise ValueError('Invalid optimization type')

    # subject to: each census tract appears in exactly one district
    for t in TRACTS:    
        m.Add(sum(x[d] * A[t,d] for d in DISTRICTS) == 1)

    # subject to: k total districts
    m.Add(sum(x[d] for d in DISTRICTS) == k)

    return m,x

In [3]:
# You do not need to pay attention to this cell but make sure you run it!
def solve(m, solver='CBC'):
    """Solve the model and specify some solver parameters."""
    if solver=='CBC':
        m.SetTimeLimit(2000000)
        params = OR.MPSolverParameters()
        params.SetDoubleParam(params.RELATIVE_MIP_GAP, 1e-4)
        status = m.Solve(params)
    elif solver=='gurobi':
        params_set = m.SetSolverSpecificParametersAsString(
                     '''TimeLimit %d
                        MIPGapAbs %d''' % (2000, 1e-4))
        if params_set:
            print('Gurobi solver parameters set successfully.')
        status = m.Solve()
    if status == OR.Solver.OPTIMAL:
        print('Optimal solution found.')
        print('Objective value =', m.Objective().Value())
    else:
        print('Optimal solution was not found - %s' % (status))